In [1]:
import pandas as pd
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv('order_brush_order.csv').drop(columns='orderid').sort_values('event_time').set_index('event_time')
df.index = pd.to_datetime(df.index)

df

,shopid,userid
event_time,,
2019-12-27 00:00:00,6042309,97707522
2019-12-27 00:00:00,104804492,97707522
2019-12-27 00:00:00,8715449,97707522
2019-12-27 00:00:02,190969466,170182475
2019-12-27 00:00:05,2859407,12532131
...,...,...
2019-12-31 23:59:43,149254894,193333760
2019-12-31 23:59:47,147941492,40258063
2019-12-31 23:59:51,187123853,2338306


In [3]:
d = {}
for shopid in df.shopid.unique():
    d[shopid] = set()

In [4]:
for shopid in tqdm(df.shopid.unique()):
    shop_df = df[df.shopid == shopid]
    ex = None
    for i, row in shop_df.iterrows():
        start = i
        if start == ex:
            continue
            
        if ex != None:
            gap = start - ex
        
        ex = start
        end = start + pd.DateOffset(hours=1)
        tmp_df = shop_df[(shop_df.index >= start) & (shop_df.index <= end)]
        
        while len(tmp_df) >= 3:
            con = len(tmp_df) / tmp_df['userid'].nunique()
            if con >= 3:
                users = tmp_df.userid.value_counts()
                d[shopid].update(list(users[users == users.max()].index))
            if end - tmp_df.index.max() < gap:
                tmp_df = tmp_df.iloc[:-1]
            else:
                break

In [5]:
out = {'shopid': [], 'userid': []}
for shopid in d:
    out['shopid'] += [shopid]
    if d[shopid] == set():
        out['userid'] += ['0']
    else:
        out['userid'] += ['&'.join(map(str, sorted(list(d[shopid]))))]

In [6]:
xx = pd.DataFrame(out)
xx.to_csv('out.csv', index=False)